In [ ]:
import os
from collections import Counter
from pathlib import Path
from PIL import Image
import numpy as np
import math
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import hashlib


In [ ]:
from pathlib import Path

# 1. Trouve dynamiquement la racine du projet (contenant .gitignore)
cwd = Path.cwd()
PROJECT_ROOT = next(p for p in (cwd, *cwd.parents) if (p / ".gitignore").exists())

print("PROJECT_ROOT =", PROJECT_ROOT)

In [ ]:
# root_dir complet
root_dir = PROJECT_ROOT/"dataset"/"plantvillage"/"data"/"plantvillage dataset"/"segmented"

# root_dir_5_img = PROJECT_ROOT/"dataset"/"plantvillage"/"data"/"plantvillage_5images"/"segmented"

In [ ]:
def is_image_valid(Image_Path: str) -> bool:
    """Vérifie que l’image n’est pas corrompue."""
    try:
        with Image.open(Image_Path) as img:
            img.verify()
        return True
    except:
        return False

Certaines images sont pratiquement noires. Elles sont identifiées et supprimées

In [ ]:
# Fonction de détection d’images quasi noires
def is_black_image(Image_Path: str, threshold: float = 5) -> bool:
    """
    Retourne True si l'image est quasiment noire.
    - Convertit en niveaux de gris et calcule la moyenne des pixels.
    - Si la moyenne est < threshold (0–255), on considère l'image noire.
    """
    try:
        with Image.open(Image_Path).convert("L") as img_gray:
            arr = np.array(img_gray)
        return arr.mean() < threshold
    except:
        # En cas d’erreur de lecture, on ne la supprime pas ici
        return False

In [ ]:
def collect_image_metadata(root_dir: str):
    """
    Parcourt `root_dir` (structure PlantVillage/segmented) et renvoie un DataFrame avec :
        - nom_plante   : nom de l’espèce (avant les trois underscores)
        - nom_maladie  : nom de la maladie ou "healthy"
        - Est_Saine    : True si le dossier est "healthy", False sinon
        - Image_Path   : chemin complet vers l’image
        - width_img, height_img: dimensions de l’image en pixels

    Exclut automatiquement les fichiers corrompus ou tout-noirs.
    """
    records = []
    
    for class_folder in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_folder)
        if not os.path.isdir(class_path):
            continue

        # 1. Séparer nom_plante et nom_maladie qu'à la première occurrence de "___"
        if "___" in class_folder:
            nom_plante, nom_maladie = class_folder.split("___", 1)
        else:
            # normalement pas utilisé ici
            nom_plante, nom_maladie = class_folder, "healthy"
        
        # 2. Déterminer Est_Saine
        est_saine = (nom_maladie.lower() == "healthy")
        
        # 3. Parcourir les images valides
        for fname in os.listdir(class_path):
            Image_Path = os.path.join(class_path, fname)
            
            # Filtre des images corrompues ou quasi-noires (fonctions à définir ailleurs)
            # if not (is_image_valid(Image_Path) and not is_black_image(Image_Path)):
                # continue
            
            try:
                with Image.open(Image_Path) as img:
                    w, h = img.size
                records.append({
                    "nom_plante" :  nom_plante,
                    "nom_maladie": nom_maladie,
                    "Est_Saine"  :   est_saine,
                    "Image_Path" :  Image_Path,
                    "width_img"  :       w,
                    "height_img" :      h
                })
            except:
                # passe si l'image ne peut être ouverte
                continue

    return pd.DataFrame(records)

In [ ]:
def show_black_images(df, path_col='Image_Path', threshold=5, max_display=9):

    # Filtrer les images quasi noires
    black_df = df[df[path_col].apply(lambda p: is_black_image(p, threshold))]
    n = min(len(black_df), max_display)
    
    if n == 0:
        print("Aucune image quasi noire détectée.")
        return
    
    # Préparer la grille de subplots
    cols = 3
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()
    
    # Afficher chaque image avec son chemin en titre
    for ax, (_, row) in zip(axes, black_df.iterrows()):
        img = Image.open(row[path_col]).convert("RGB")
        ax.imshow(img)
        ax.set_title(row[path_col].split('/')[-1], fontsize=8)
        ax.axis('off')
    
    # Cacher les axes vides
    for ax in axes[n:]:
        ax.axis('off')
    
    plt.suptitle(f"{n} images quasi noires (moyenne gray < {threshold})", fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
df_meta = collect_image_metadata(root_dir)
df_meta.head()

In [ ]:
df_meta.shape

In [ ]:
show_black_images(df_meta, path_col='Image_Path', threshold=5, max_display=18)

In [ ]:
#  Identification et suppression 

# Détection
df_meta['is_black'] = df_meta['Image_Path'].apply(is_black_image)
display("images quasi noires", df_meta['is_black'], df_meta["Image_Path"])

# Nombre d'images quasi noires
nb_black = df_meta['is_black'].sum()
print(f"{nb_black} images quasi noires détectées.")

# Supprimer ces lignes de df
df_clean = df_meta[~df_meta['is_black']].reset_index(drop=True)

# Vérification
print(f"DataFrame nettoyé : {len(df_clean)}/{len(df_meta)} images restantes.")

Identification des doublons d'images

In [ ]:
def find_duplicate_image_paths(df, path_col='Image_Path', md5_col='md5'):
    """
    1. Ajoute une colonne md5 au DataFrame en calculant le hash MD5 du contenu binaire de chaque image.
    2. Identifie tous les groupes d'images dont le md5 est dupliqué.

    Paramètres :
        df (pd.DataFrame)   : DataFrame contenant au moins `path_col`
        path_col (str)      : nom de la colonne avec les chemins d’images
        md5_col (str)       : nom de la colonne à créer pour le hash

    Retour :
        df_with_md5 (pd.DataFrame) : DataFrame original enrichi de `md5_col`
        dup_groups (pd.DataFrame)  : DataFrame à deux colonnes :
            - md5_col : la valeur du hash dupliqué
            - duplicates : liste des chemins d’images ayant ce hash
    """
    df = df.copy()

    # Calculer et ajouter la colonne MD5
    def compute_md5(path):
        try:
            with open(path, 'rb') as f:
                return hashlib.md5(f.read()).hexdigest()
        except:
            return None

    df[md5_col] = df[path_col].apply(compute_md5)

    # Identifier les groupes de MD5 en double
    duplicates = (
        df[df.duplicated(md5_col, keep=False)]
        .groupby(md5_col)[path_col]
        .apply(list)
        .reset_index(name='duplicates')
    )

    return df, duplicates

In [ ]:
# Identification des doublons par espèce et par maladie
def plot_duplicate_distribution_by_class(df, md5_col='md5'):
    """
    Vérifie et affiche la distribution des doublons d'images par espèce et par maladie dans le DataFrame.
    
    df doit contenir :
      - une colonne md5_col avec le hash MD5 de chaque image
      - 'nom_plante' et 'nom_maladie'
    """
    # Filtrer les lignes dupliquées (au moins 2 images identiques)
    dup_df = df[df.duplicated(md5_col, keep=False)].copy()
    
    # Nombre total d'images dupliquées par espèce
    species_dup_images = (
        dup_df['nom_plante']
        .value_counts()
        .reset_index(name='duplicate_image_count')
        .rename(columns={'index': 'nom_plante'})
    )
    fig2 = px.bar(
        species_dup_images,
        x='nom_plante',
        y='duplicate_image_count',
        title="Images dupliquées par espèce",
        labels={'nom_plante': 'Espèce', 'duplicate_image_count': "Nombre d'images dupliquées"},
    )
    fig2.update_layout(xaxis_tickangle=-45)
    fig2.show()
    
    # Même type de graphique pour la maladie
    
    disease_dup_images = (
        dup_df['nom_maladie']
        .value_counts()
        .reset_index(name='duplicate_image_count')
        .rename(columns={'index': 'nom_maladie'})
    )
    fig4 = px.bar(
        disease_dup_images,
        x='nom_maladie',
        y='duplicate_image_count',
        title="Images dupliquées par maladie",
        labels={'nom_maladie': 'Maladie', 'duplicate_image_count': "Nombre d'images dupliquées"},
    )
    fig4.update_layout(xaxis_tickangle=-45)
    fig4.show()

In [ ]:
# Enrichir df avec la colonne 'md5' et récupérer les groupes de doublons
df_with_md5, dup_groups = find_duplicate_image_paths(
    df_clean, 
    path_col='Image_Path',  
    md5_col='md5'
)

print("Nombre total de MD5 uniques :", df_with_md5['md5'].nunique())
print("Groupes de doublons trouvés :\n", dup_groups.head())

# Tracer la distribution des doublons par espèce et par maladie
plot_duplicate_distribution_by_class(df_with_md5, md5_col='md5')

In [ ]:
# Supprime tous les doublons MD5 en ne gardant que le premier exemplaire
df_raw_data = df_with_md5.drop_duplicates(subset='md5', keep='first').reset_index(drop=True)

In [ ]:
# vérifications
# Taille
print(f"Images avant nettoyage : {len(df_clean)}")
print(f"Images après   nettoyage : {len(df_raw_data)}")

# Absence de doublons
assert df_raw_data['md5'].nunique() == len(df_raw_data)


In [ ]:
df_raw_data.columns

Explotation du dataframe "df_raw_data" sans image corrompue, sans image quasi noire et sabs doublons

In [ ]:
def plot_dataset_saine_classes_distribution(df: pd.DataFrame):
    """
    Affiche la distribution du nombre d’images par classe 'espèce_saine'
    **uniquement pour les feuilles saines** (on exclut les images malades).
    """
    # Filtrer pour ne conserver que les images malades
    df_saines = df[df['Est_Saine'] == True].copy()

    # Construire la colonne 'classe' = "nom_plante_nom_maladie"
    df_saines['classe'] = df_saines['nom_plante'] + "_healthy"

    # Compter le nombre d’images par classe
    rep = (
        df_saines
        .groupby('classe')
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)
    )

    # Tracer le bar chart
    fig = px.bar(
        rep,
        x='classe',
        y='count',
        title="Distribution des classes  saines",
        labels={'classe': "Classe (espèce saine)", 'count': "Nombre d’images"},
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis={'categoryorder': 'total descending'},
        margin={'t':50, 'b':150}
    )
    fig.show()

def plot_dataset_disease_classes_distribution(df: pd.DataFrame):
    """
    Affiche la distribution du nombre d’images par classe 'espèce_maladie'
    **uniquement pour les feuilles malades** (on exclut les images saines).
    """
    # Filtrer pour ne conserver que les images malades
    df_malades = df[df['Est_Saine'] == False].copy()

    # Construire la colonne 'classe' = "nom_plante_nom_maladie"
    df_malades['classe'] = df_malades['nom_plante'] + "_" + df_malades['nom_maladie']

    # Compter le nombre d’images par classe
    rep = (
        df_malades
        .groupby('classe')
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)
    )

    # Tracer le bar chart
    fig = px.bar(
        rep,
        x='classe',
        y='count',
        title="Distribution des classes  malades",
        labels={'classe': "Classe (espèce)", 'count': "Nombre d’images"},
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis={'categoryorder': 'total descending'},
        margin={'t':50, 'b':150}
    )
    fig.show()



def plot_class_distribution(df: pd.DataFrame):
    """Nombre d’images par espèce."""
    counts = df['nom_plante'].value_counts().reset_index()
    counts.columns = ['nom_plante', 'count']
    fig = px.bar(counts, x='nom_plante', y='count',
                 title="Nombre d'images par espèce",
                 labels={'nom_plante':'Espèce', 'count':'Nombre d’images'})
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

def plot_dataset_classes_distribution(df: pd.DataFrame):
    """
    Affiche la distribution du nombre d’images par classe dans PlantVillage Segmented.
    # Préparer la colonne 'classe'
    """
    ds = (
        df
        .assign(
            classe=lambda d: d.apply(
                lambda r: f"{r['nom_plante']}_{r['nom_maladie']}"
                          if r['nom_maladie'] and r['nom_maladie'] != "Aucune"
                          else r['nom_plante'],
                axis=1
            )
        )
    )
    # Compter le nombre d’images par classe
    rep = ds.groupby('classe').size().reset_index(name='count')

    # Tracer le bar chart
    fig = px.bar(
        rep,
        x='classe',
        y='count',
        title="Distribution des classes ",
        labels={'classe': "Classe (espèce)", 'count': "Nombre d’images"},
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis={'categoryorder':'total descending'},
        margin={'t':50, 'b':150}
    )
    fig.show()

def plot_size_scatter(df: pd.DataFrame):
    """Scatter width_img vs height_img."""
    fig = px.scatter(df, x='width_img', y='height_img',
                     title="Scatter : largeur vs hauteur",
                     labels={'width_img':'Largeur (px)', 'height_img':'Hauteur (px)'},
                     opacity=0.5)
    fig.show()

def plot_size_histogram(df: pd.DataFrame):
    """Histogrammes superposés des largeurs et hauteurs."""
    df2 = df.melt(id_vars=['Image_Path','nom_plante'], value_vars=['width_img','height_img'],
                  var_name='Dimension', value_name='Pixels')
    fig = px.histogram(df2, x='Pixels', color='Dimension',
                       barmode='overlay', nbins=30,
                       title="Distribution des largeurs et hauteurs",
                       labels={'Pixels':'Pixels','Dimension':'Type'})
    fig.update_traces(opacity=0.6)
    fig.show()

def plot_health_distribution_by_plant(df: pd.DataFrame):
    """
    Bar chart groupé : pour chaque espèce, nombre d'images saines vs malades.
    """
    rep = df.groupby(['nom_plante','Est_Saine']).size().reset_index(name='count')
    rep['Etat'] = rep['Est_Saine'].map({True:'Saine', False:'Malade'})
    fig = px.bar(rep, x='nom_plante', y='count', color='Etat', barmode='group',
                 title="Répartition des images saines vs malades par espèce",
                 labels={'nom_plante':'Espèce', 'count':'Nombre d’images'})
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [ ]:
def show_sample_images(df: pd.DataFrame, n_per_class:int=3, thumb_size=(100,100)):
    """Affiche n_per_class miniatures par espèce."""
    import matplotlib.pyplot as plt
    classes = df['nom_plante'].unique()
    fig, axes = plt.subplots(len(classes), n_per_class, figsize=(n_per_class*2, len(classes)*2))
    for i, cls in enumerate(classes):
        paths = df[df['nom_plante']==cls]['Image_Path'].sample(n_per_class, random_state=0).tolist()
        for j, p in enumerate(paths):
            img = Image.open(p).resize(thumb_size)
            ax = axes[i,j] if len(classes)>1 else axes[j]
            ax.imshow(img); ax.axis('off')
        axes[i,0].set_ylabel(cls, rotation=0, labelpad=40, va='center')
    plt.suptitle("Exemples par espèce", y=0.92)
    plt.tight_layout(); plt.show()


In [ ]:
if __name__ == "__main__":
    # df_meta = collect_image_metadata(root_dir)

    print("Total images valides :", len(df_raw_data))
    plot_class_distribution(df_raw_data)
    plot_dataset_classes_distribution(df_raw_data)
    plot_health_distribution_by_plant(df_raw_data)
    plot_dataset_saine_classes_distribution(df_raw_data)
    plot_dataset_disease_classes_distribution(df_raw_data)
    plot_size_scatter(df_raw_data)
    plot_size_histogram(df_raw_data)



    # Afficher quelques exemples
    show_sample_images(df_raw_data, n_per_class=3)


In [ ]:
# Vérification du nombre de classes
folders = sorted(os.listdir(root_dir))
print(f"Nombre total de classes : {len(folders)}\n")
for f in folders:
    print(f)

In [ ]:
# Sauvegarde du dataframe df_raw_data en fichier .csv
df_raw_data.to_csv("Plant_V_Seg_clean.csv", index=False)

In [ ]:
df = pd.read_csv('Plant_V_Seg_clean.csv')
df.columns


In [ ]:
# Liste des espèces et organisation du grid
species = df['nom_plante'].unique()
n = len(species)
cols = 4
rows = math.ceil(n / cols)

fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
axes = axes.flatten()

for ax, sp in zip(axes, species):
    # Récupère le premier chemin d'image pour cette espèce
    img_path = df.loc[df['nom_plante'] == sp, 'Image_Path'].iat[0]
    img = Image.open(img_path)
    ax.imshow(img)
    ax.set_title(sp, fontsize=10)
    ax.axis('off')

# Masquer les axes supplémentaires s’il y en a
for ax in axes[n:]:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 2) Filtrer pour ne garder que Corn
corn_df = df[df['nom_plante'] == 'Corn_(maize)']

# 3) Tirage aléatoire de 16 échantillons
import math
sample_count = min(16, len(corn_df))
sample_df = corn_df.sample(n=sample_count, random_state=42)

# 4) Affichage en grille 4×4
import matplotlib.pyplot as plt
from PIL import Image

cols = 4
rows = math.ceil(sample_count / cols)
fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
axes = axes.flatten()

for ax, (_, row) in zip(axes, sample_df.iterrows()):
    img = Image.open(row['Image_Path'])
    ax.imshow(img)
    ax.axis('off')

for ax in axes[sample_count:]:
    ax.axis('off')

plt.tight_layout()
plt.show()
